In [1]:
import random
import copy

In [2]:
##Calculate euclidean distance for any two points with multivariables 
def euclideanDistance(point1 = [], point2 = []): 
    if len(point1) != len(point2):
        raise Exception("List lenght doesn't match")
    else: 
        i = 0
        squareSum= 0
        eucliDistance = 0
        while i < len(point1):
            squareSum = squareSum +pow(point1[i] - point2[i],2)
            i = i+1
        eucliDistance = pow(squareSum, 0.5)
        return eucliDistance

In [3]:
##Calculate manhattan distance for any two points with multivariables 
def manhattanDistance(point1 = [], point2= []):
    if len(point1) != len(point2):
        raise Exception("List lenght doesn't match")
    else:
        ##Calculate 
        i = 0
        blockSum = 0
        while i < len(point1):
            blockSum = blockSum + abs(point1[i] - point2[i])
            i=i+1
        return blockSum         

In [4]:
##Treat samples in our file when there is a unknown value for any variable 
##This unknown value should be represented as a '?'
def treatSamples(filepath):
    file = open(filepath, "r")
    lines = file.readlines()
    file.close()
    file_treated = filepath + ".treated"
    samples_retired = 0
    file2 = open(file_treated, "a")
    for line in lines:
        if "?" not in line:
            file2.write(line)
        else:
            samples_retired += 1
    file2.close()
    print("Samples treated!!!")
    print("Samples retired=", samples_retired)

In [5]:
##Test for Euclidean distance with two points taken from our dataset
a = [2,2,0,3,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,55,2]
b = [2,2,0,3,0,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,0,1,0,0,0,0,3,0,0,0,1,0,21,2]
c = [1,0]
d = [0,1]
print("Distancia euclidiana,"+str(euclideanDistance(a,b)))
print("Distancia euclidiana,"+str(euclideanDistance(c,d)))
print("Distancia manhattan,"+str(manhattanDistance(a,b)))
print("Distancia manhattan,"+str(manhattanDistance(c,d)))

Distancia euclidiana,34.10278580995987
Distancia euclidiana,1.4142135623730951
Distancia manhattan,39
Distancia manhattan,2


In [6]:
#Treating data
treatSamples("dermatology.data")

Samples treated!!!
Samples retired= 8


In [4]:
##Turning our data .treated file into an list of lists where the value of these lists are the values of each sample

def turnIntoMatrix(filepath):
    file = open(filepath, "r")
    lines = file.readlines()
    file.close()
    
    ##Preparing the list of lists 
    matrix = []
    
    for line in lines:
        ##sample will store all the values found 
        ##in a line which represents a sample when the string values were turned into numbers 
        sample = []
        line = line.replace("\n", "")
        split = line.split(",")
        for value in split:
            sample.append(float(value))
        matrix.append(sample)
    return matrix 

In [7]:
##Test turnIntoMatrix
##parsing the .treated file to create our sample matrix 
matrix = turnIntoMatrix("dermatology.data.treated")
##Printing first and second lines of our matrix 
print("Fist Sample", matrix[0])
print("Second Sample", matrix[1])

Fist Sample [2.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 55.0, 2.0]
Second Sample [3.0, 3.0, 3.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 8.0, 1.0]


In [9]:
##A simple way to find the closest neighbours 
def getNeighbour(typeofdistance, matrix, sample, neighbours):
    distance = []
    euclidean = "euclidean"
    manhattan = "manhattan"
    if typeofdistance == euclidean:
        for row in matrix:
            euclidean = euclideanDistance(sample[:34], row[:34])
            label = row[34]
            distance.append([euclidean, label])
    elif typeofdistance == manhattan:
        for row in matrix:
            manhattan = manhattanDistance(sample[:34], row[:34])
            label = row[34]
            distance.append([manhattan, label])
    distance.sort(key = lambda x: x[0])
    return distance[:neighbours]
# Class code:   Class:                  Number of instances:
#       1             psoriasis			    112
#       2             seboreic dermatitis             61
#       3             lichen planus                   72
#       4             pityriasis rosea                49
#       5             cronic dermatitis               52    
#       6             pityriasis rubra pilaris        20
# Choose the class winner for sample test by counting the maximum of votes of a class 
def byVote(distances):
    classes = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0
    }
    for distance in distances:
        if distance[1] == 1.0:
            classes[1] += 1
        elif distance[1] == 2.0:
            classes[2] += 1
        elif distance[1] == 3.0:
            classes[3] += 1
        elif distance[1] == 4.0: 
            classes[4] += 1
        elif distance[1] == 5.0:
            classes[5] += 1
        elif distance[1] == 6.0:
            classes[6] += 1
        else: 
            raise Exception("Value not recognizable")
    votes = max(classes.values())
    winner = max(classes, key=classes.get)
    return (winner, votes) 

def prediction(traingroup, testsample, typeofdistance, knn):
    distances = getNeighbour(typeofdistance, traingroup, testsample, knn)
    (winner, votes) = byVote(distances)
    return (winner, votes)
##Return a list of samples randomly choosed accordingly with a given percentage 
def getRandomDataSet(matrix, percentage):
    numberOfSamples = int(round(percentage*len(matrix)))
    return random.sample(matrix, numberOfSamples)
                         
                        

In [10]:
print(matrix[3])
getNeighbour("euclidean", matrix, matrix[3], 15)
distances = getNeighbour("euclidean", matrix, matrix[3], 15)
print(distances)
byVote(distances)

[2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 2.0, 0.0, 3.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 40.0, 1.0]
[[0.0, 1.0], [4.898979485566356, 1.0], [5.385164807134504, 1.0], [5.477225575051661, 1.0], [5.5677643628300215, 1.0], [5.744562646538029, 1.0], [5.744562646538029, 1.0], [5.830951894845301, 1.0], [5.916079783099616, 1.0], [5.916079783099616, 1.0], [6.0, 1.0], [6.0, 1.0], [6.244997998398398, 1.0], [6.324555320336759, 1.0], [6.48074069840786, 1.0]]


(1, 15)

In [11]:
trainingSet = getRandomDataSet(matrix, 0.8)
testSet = getRandomDataSet(matrix, 0.2)
print(trainingSet)
print(len(testSet))

[[2.0, 1.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 36.0, 5.0], [3.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 0.0, 2.0, 3.0, 2.0, 3.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 35.0, 1.0], [2.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 55.0, 2.0], [3.0, 3.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 3.0, 3.0, 3.0, 2.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 30.0, 1.0], [2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 40.0, 1.0], [2.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0


72


In [12]:
def matrizDeConfusao(matrix, typeofdistance, knn):
    matriz = [[0,0,0,0,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,0],
              [0,0,0,0,0,0]]
    trainingSet = getRandomDataSet(matrix, 0.8)
    testSet = getRandomDataSet(matrix, 0.2)
    for sample in testSet:
        winner, votes = prediction(trainingSet, sample, typeofdistance, knn)
        if sample[34] == winner:
            prevision = int(winner) - 1
            matriz[prevision][prevision] += 1
        else:
            trueClass = int(sample[34]) - 1
            prevision = int(winner) - 1
            matriz[prevision][trueClass] += 1
        
    return matriz
    
def accuracyTotal(matriz):
    somaAcertos = 0
    somaTotal = 0
    for i in range(len(matriz)):
        somaAcertos += matriz[i][i]
    for i in range(len(matriz)):
        for j in range(len(matriz[i])):
            somaTotal += matriz[i][j]
    return somaAcertos/somaTotal

def accuracyByClass(matriz, classe): 
    TP = matriz[classe-1][classe-1]
    Aux = createSubMatrix(matriz, classe, classe)
    TN = 0
    for i in range(len(Aux)):
        TN += sum(Aux[i])
    FP = sum(matriz[classe-1])-matriz[classe-1][classe-1]
    FN = sum(column(matriz,classe-1))-matriz[classe-1][classe-1]
    #print(TP, TN, FP, FN)
    #print("classe", classe,  (TP+TN)/(TP+TN+FP+FN))
    return  (TP+TN)/(TP+TN+FP+FN)

def column(matriz, classe):
    coluna = list()
    for line in matriz:
        coluna.append(line[classe])
    return coluna    
def createSubMatrix(matriz, line, column):
    sub = copy.deepcopy(matriz)
    sub.remove(sub[line-1])
    submatrix = list()
    for line in sub:
        submatrix.append(line.remove(line[column-1]))
    return sub
def testKNN(matrix, typeofdistance):
    acuraciaMelhorCaso = 0
    acuraciaPiorCaso = 0
    matrizMelhorCaso = []
    acuraciaMediaTotal = 0
    acuraciaClasse1 = 0
    acuraciaClasse2 = 0 
    acuraciaClasse3 = 0 
    acuraciaClasse4 = 0 
    acuraciaClasse5 = 0 
    acuraciaClasse6 = 0
    for i in range(100):
        confusao = matrizDeConfusao(matrix, typeofdistance, 3)
        acuracia = accuracyTotal(confusao)
        acuraciaMediaTotal += acuracia
        acuraciaClasse1 += accuracyByClass(confusao, 1)
        acuraciaClasse2 += accuracyByClass(confusao, 2)
        acuraciaClasse3 += accuracyByClass(confusao, 3)
        acuraciaClasse4 += accuracyByClass(confusao, 4)
        acuraciaClasse5 += accuracyByClass(confusao, 5)
        acuraciaClasse6 += accuracyByClass(confusao, 6)
        if i == 0:
            acuraciaMelhorCaso = acuracia
            acuraciaPiorCaso = acuracia
        else:
            acuracia = accuracyTotal(confusao)
            if acuraciaMelhorCaso < acuracia:
                acuraciaMelhorCaso = acuracia
                matrizMelhorCaso = confusao
            if acuraciaPiorCaso > acuracia:
                acuraciaPiorCaso = acuracia
                matrizPiorCaso = confusao
    print("Acuracia total média", acuraciaMediaTotal/100)
    print("Acuracia média classe 1", acuraciaClasse1/100)
    print("Acuracia média classe 2", acuraciaClasse2/100)
    print("Acuracia média classe 3", acuraciaClasse3/100)
    print("Acuracia média classe 4", acuraciaClasse4/100)
    print("Acuracia média classe 5", acuraciaClasse5/100)
    print("Acuracia média Classe 6", acuraciaClasse6/100)
    print("Acuracia do pior caso", acuraciaPiorCaso)
    print("#######MATRIZ DE CONFUSÃO DO PIOR CASO#######")
    for line in matrizPiorCaso:
        print(str(line))
    print("Acuracia melhor caso", acuraciaMelhorCaso)
    print("#######MATRIZ DE CONFUSÃO DO MELHOR CASO#######")
    for line in matrizMelhorCaso:
        print(str(line))
        
        


In [13]:
testKNN(matrix, "euclidean")
testKNN(matrix, "manhattan")

Acuracia total média 0.941111111111111
Acuracia média classe 1 0.9938888888888897
Acuracia média classe 2 0.9506944444444446
Acuracia média classe 3 0.9995833333333334
Acuracia média classe 4 0.9558333333333334
Acuracia média classe 5 0.9890277777777792
Acuracia média Classe 6 0.9931944444444453
Acuracia do pior caso 0.875
#######MATRIZ DE CONFUSÃO DO PIOR CASO#######
[24, 0, 0, 0, 0, 0]
[0, 9, 0, 3, 2, 0]
[0, 0, 11, 0, 0, 0]
[0, 2, 0, 6, 1, 0]
[0, 0, 0, 0, 7, 0]
[0, 1, 0, 0, 0, 6]
Acuracia melhor caso 1.0
#######MATRIZ DE CONFUSÃO DO MELHOR CASO#######
[27, 0, 0, 0, 0, 0]
[0, 11, 0, 0, 0, 0]
[0, 0, 18, 0, 0, 0]
[0, 0, 0, 6, 0, 0]
[0, 0, 0, 0, 9, 0]
[0, 0, 0, 0, 0, 1]
Acuracia total média 0.9637500000000007
Acuracia média classe 1 0.9980555555555558
Acuracia média classe 2 0.9650000000000009
Acuracia média classe 3 1.0
Acuracia média classe 4 0.9711111111111117
Acuracia média classe 5 0.9954166666666673
Acuracia média Classe 6 0.997916666666667
Acuracia do pior caso 0.9027777777777778


33
